In [63]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os
import glob
import pandas as pd

In [78]:
from dotenv import load_dotenv
import os

load_dotenv()  # Load .env file

id = os.getenv("linkedin_id")
pw = os.getenv("linkedin_pw")



period = 'today'
# period = 'past week'

job_title = 'data scientist'

pages = 10

In [65]:
analyst_keywords = ['data analyst','business analyst','analyst','tableau','power bi','business intelligence']

scientist_keywords = ['data scientist',
             'machine learning','machine learning engineer','ml engineer',
             'ai engineer','artificial intelligence','llm engineer'
             'gen ai','gen ai engineer','generative ai']

senior_keywords = ['Senior','senior', 'Sr.']

top_keyworkds = ['principal','staff','lead']

analyst_pattern = '|'.join(analyst_keywords)
scientist_pattern = '|'.join(scientist_keywords)
senior_pattern = '|'.join(senior_keywords)
top_pattern = '|'.join(top_keyworkds)


sponsorship = ['sponsorship','visa']
yes_sponsor = ['will consider sponsor','will sponsor', 'open to sponsor', 'sponsorship is available']
citizen = ['US citizen, U.S. citizen', 'clearance']

sponsorship_pattern = '|'.join(sponsorship)
citizen_pattern = '|'.join(citizen)
yes_pattern = '|'.join(yes_sponsor)


def get_job_details(job):
    title = job.find_element(By.TAG_NAME, "strong").text
    company = job.find_element(By.CLASS_NAME, "artdeco-entity-lockup__subtitle").find_element(By.TAG_NAME, "span").text
    where = job.find_element(By.CLASS_NAME, "artdeco-entity-lockup__caption").find_element(By.TAG_NAME, "span").text
    try:
        easy_apply = job.find_element(By.CLASS_NAME, "job-card-list__footer-wrapper").find_element(By.TAG_NAME, "span").text
        if easy_apply != 'EasyApply':
            easy_apply = "Apply"
    except:
        easy_apply = "Apply"


    link_block = driver.find_element(By.CLASS_NAME, "jobs-details__main-content")
    link_area = link_block.find_element(By.CSS_SELECTOR, "div.t-24.job-details-jobs-unified-top-card__job-title")
    link = link_area.find_element(By.TAG_NAME, "a").get_attribute("href")

    # job_description = driver.find_element(By.CLASS_NAME, "jobs-description__container").text

    job_description = driver.find_element(By.CLASS_NAME, "jobs-description__container").text


    sponsor_mentioned = 1 if re.search(sponsorship_pattern, job_description, re.IGNORECASE) else 0
    will_sponsor = 1 if re.search(yes_pattern, job_description, re.IGNORECASE) else 0
    clearance_mentioned = 1 if re.search(citizen_pattern, job_description, re.IGNORECASE) else 0

    data = {
        'job_title': title,
        'company': company,
        'location': where,
        'link': link,
        'apply': easy_apply,
        # 'description': job_description
        'sponsorship': sponsor_mentioned,
        'will_sponsor': will_sponsor,
        'clearance': clearance_mentioned
    }
    
    return data

import re

In [ ]:
def format_job_title(job_title):
    words = job_title.split()
    formatted_title = words[0]  # Keep the first word as is
    for word in words[1:]:
        formatted_title += "%20" + word  # Add "%20" before each subsequent word
    return formatted_title

job_search = format_job_title(job_title)

driver = webdriver.Chrome()

# Open LinkedIn login page
driver.get("https://www.linkedin.com/login")

# Wait for the login page to load
try:
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "rememberMeOptIn-checkbox")))

    checkbox = driver.find_element(By.ID, "rememberMeOptIn-checkbox")

    # Check if it is already selected
    # Use JavaScript to set the 'checked' property to false
    driver.execute_script("arguments[0].checked = false;", checkbox)
except:
    pass

# Enter your login credentials
username = driver.find_element(By.ID, "username")
password = driver.find_element(By.ID, "password")
username.send_keys(id)  # Replace with your LinkedIn email
password.send_keys(pw)          # Replace with your LinkedIn password

# Click the login button
password.send_keys(Keys.RETURN)

# Wait for redirection after login
WebDriverWait(driver, 10).until(EC.url_contains("https://www.linkedin.com/feed/"))

if period == "today":
    time_filter = 'r86400'
elif period == 'past week':
    time_filter = 'r604800'

job_list = []
clearance_list = []


search_url = f"https://www.linkedin.com/jobs/search/?f_TPR={time_filter}&keywords={job_search}&origin=JOB_SEARCH_PAGE_JOB_FILTER&sortBy=DD&start=0"

driver.get(search_url)

for i in range(2,5):

    parent_list = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='main']/div/div[2]/div[1]/div"))
    )

    job_listings = parent_list.find_elements(By.XPATH, "./*/li")  # Only direct children

    for index in range(len(job_listings)):
        try:
            job_listings = driver.find_elements(By.XPATH, "//*[@id='main']/div/div[2]/div[1]/div/*/li")
            job = job_listings[index]  # Get the job element again

            job.click()
            # top.find_element(By.TAG_NAME, "a").get_adttribute("text")
            # title = job.find_element(By.TAG_NAME, "strong").text
            # Search for the pattern in the text

            # WebDriverWait(job, 20).until(
            #             EC.visibility_of_all_elements_located((By.XPATH, '//*[@id="main"]/div/div[2]/div[2]/div/div[2]/div/div/div[1]/div/div[4]/article'))
            #         )
            
            WebDriverWait(job, 20).until(
                        EC.visibility_of_all_elements_located((By.XPATH,'//*[@id="job-details"]/div/p'))
                    )



            title = WebDriverWait(job, 20).until(
                EC.presence_of_element_located((By.TAG_NAME, "strong"))
            ).text


            if not re.search(top_pattern, title, re.IGNORECASE):
                if re.search(analyst_pattern, title, re.IGNORECASE):

                    data = get_job_details(job)
                    if data['clearance'] == 1:
                        clearance_list.append(data)
                    else:
                        print(data['job_title'])
                        job_list.append(data)

                elif re.search(scientist_pattern, title, re.IGNORECASE):
                    if not re.search(senior_pattern, title, re.IGNORECASE):

                        data = get_job_details(job)
                        if data['clearance'] == 1:
                            clearance_list.append(data)
                        else:
                            print(data['job_title'])
                            job_list.append(data)

                else:
                    pass
        except:
            pass

    page_button = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, f"button[aria-label='Page {i}']"))
    )

    # page_button = driver.find_element(By.CSS_SELECTOR, f"button[aria-label='Page {i}']")
    page_button.click()

# driver.quit()

Compliance Systems/Data Analyst I – Surveillance Solutions
Data Tools Admin Support/Analyst
Data Scientist – Health and Risk Solutions
Fullstack AI Engineer
Data Scientist & Experimentation Specialist (ML-Driven Pricing)
Senior Business Applications Analyst
AI Engineer
Power BI / SSRS Pagianted Report Developer
Business Systems Analyst - Contact Center Solutions: 25-03839
Sr. GIS Analyst/Data Architect
Business System Analyst
Business Analyst (Process Improvement & Agile Focus) 🔍
Senior Business Configuration Analyst
Data Scientist
Data Analytics / Business Intelligence, Transaction Banking, Data Platform
Machine Learning Engineer - Healthcare & MedTech
Marketing Data Scientist
Data Analyst (Logistics)
Digital Marketing Analyst with Tableau or PowerBI
Strategic Sales Development Analyst (Health Care & Technology)
Data Scientist – Health and Risk Solutions
Data Scientist
Contact Center AI Automation Analyst
Configuration Analyst I
Program Analyst 1 (Cannabis) - VID 180534
Data Analyst (

In [67]:
import pandas as pd
pd.DataFrame(job_list)

,job_title,company,location,link,apply,sponsorship,will_sponsor,clearance
0,Compliance Systems/Data Analyst I – Surveillan...,Stifel Financial Corp.,"St Louis, MO (On-site)",https://www.linkedin.com/jobs/view/4149256821/...,Apply,0,0,0
1,Data Tools Admin Support/Analyst,The Select Group,"Raleigh, NC (On-site)",https://www.linkedin.com/jobs/view/4146784453/...,Apply,0,0,0
2,Data Scientist – Health and Risk Solutions,Sun Life,"Nashville, TN",https://www.linkedin.com/jobs/view/4149266431/...,Apply,0,0,0
3,Fullstack AI Engineer,techolution,"Ridgewood, NJ (On-site)",https://www.linkedin.com/jobs/view/4149183631/...,Apply,0,0,0
4,Data Scientist & Experimentation Specialist (M...,MDA Edge,"San Jose, CA (Remote)",https://www.linkedin.com/jobs/view/4146783573/...,Apply,0,0,0
5,Senior Business Applications Analyst,Quanex,"Minneapolis, MN (On-site)",https://www.linkedin.com/jobs/view/4146783573/...,Apply,0,0,0
6,AI Engineer,Markerr,New York City Metropolitan Area (Remote),https://www.linkedin.com/jobs/view/4149263764/...,Apply,0,0,0
7,Power BI / SSRS Pagianted Report Developer,CREx,New York City Metropolitan Area (Remote),https://www.linkedin.com/jobs/view/4149263764/...,Apply,0,0,0
8,Business Systems Analyst - Contact Center Solu...,"Akraya, Inc.","San Jose, CA (Remote)",https://www.linkedin.com/jobs/view/4149261712/...,Apply,0,0,0
9,Sr. GIS Analyst/Data Architect,UDC,"Washington, DC (On-site)",https://www.linkedin.com/jobs/view/4146779985/...,Apply,0,0,0


In [ ]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font
from openpyxl.worksheet.hyperlink import Hyperlink

df = pd.DataFrame(data)

# Save initial DataFrame to an Excel file
excel_file = "jobs.xlsx"
df.to_excel(excel_file, index=False)

# Load the workbook to modify hyperlinks
wb = load_workbook(excel_file)
ws = wb.active

# Add hyperlinks to the 'apply' column (5th column, assuming 1-based index)
for row in range(2, len(df) + 2):  # Start from the second row (after headers)
    cell = ws.cell(row=row, column=5)  # 'apply' column
    cell.hyperlink = df.loc[row - 2, 'link']  # Add hyperlink
    cell.font = Font(color="0000FF", underline="single")  # Style the hyperlink
    cell.value = "Apply Here"  # Optional: Change the cell display text

# Save the updated workbook
wb.save(excel_file)

In [70]:
job_list[10]

{'job_title': 'Business System Analyst',
 'company': 'Upsilon IT Solutions',
 'location': 'Chicago, IL (On-site)',
 'link': 'https://www.linkedin.com/jobs/view/4149262502/?alternateChannel=search&refId=HuyWmzAdMqZzi3krk6Z1rw%3D%3D&trackingId=6o8hTnxtUqNjczlmF2ZWCw%3D%3D&trk=d_flagship3_search_srp_jobs',
 'apply': 'Apply',
 'sponsorship': 1,
 'will_sponsor': 0,
 'clearance': 0}

In [77]:
# Open the markdown file in write mode
with open("job_postings.md", "w") as md_file:
    for job in job_list:
        # Generate markdown for each job
        markdown_content = f"""
# {job['job_title']}

**Company:** {job['company']}  

**Location:** {job['location']}

**Applicaiton:** {job['apply']}

**[Apply Now]({job['link']})**  

**Sponsorship:** {job['sponsorship']}

**Will sponsor:** {job['will_sponsor']}

---

<br><br><br><br>
"""
        # Write the markdown content to the file
        md_file.write(markdown_content)

In [74]:
import webbrowser

url = [i['link'] for i in job_list]

for i in url:
    webbrowser.open(i)



In [20]:
import json

# Save as a JSON file
with open("analyst.json", "w") as file:
    json.dump(analyst_list, file, indent=4)

# Save as a JSON file
with open("scientist.json", "w") as file:
    json.dump(scientist_list, file, indent=4)

In [ ]:
import json

# Open and read the JSON file
with open("analyst.json", "r") as file:
    data = json.load(file)  # Parse the JSON file content



[{'job_title': 'Data Scientist',
  'company': 'Umicas',
  'location': 'United States (Remote)',
  'link': 'https://www.linkedin.com/jobs/view/4145886551/?alternateChannel=search&refId=xo5WmOyo51ZboFqf8zC%2BDQ%3D%3D&trackingId=k2zaulASrDzyAu0CaHkuZg%3D%3D&trk=d_flagship3_search_srp_jobs',
  'apply': 'Apply',
  'description': 'About the job\nSenior UI/UX Developer (Remote - US - PST)Shape the Future of AI-Driven TechnologyCompensation: Starting at $333,700 per year (equivalent to an hourly rate of approximately $160.43 based on a 40-hour work week). No overtime billable hours.Location: Remote (US - Pacific Standard Timezone Available Required)Employment Type: Contract (W2 or 1099)About the Company:Headquartered in Cupertino, our client is a leading technology company developing next-generation AI-driven technologies. They are driven by a mission to redefine user experiences, delivering more relevant, personalized, and efficient solutions. As they expand, they are seeking a talented Senio

In [ ]:
# Open the markdown file in write mode
with open("job_postings.md", "w") as md_file:
    for job in job_list:
        # Generate markdown for each job
        markdown_content = f"""
# {job['job_title']}

**Company:** {job['company']}  

**Location:** {job['location']}

**Applicaiton:** {job['apply']}

**[Apply Now]({job['link']})**  

## Job Description
{job['description']}

---

"""
        # Write the markdown content to the file
        md_file.write(markdown_content)

In [42]:
test = analyst_list[0]['description']

In [53]:
test

"About the job\nThe Support Analyst is responsible for delivering procedural support to all clients through telephone, email, chat, and client portal correspondence. The scope of this job centers around timely resolution of escalated client issues involving a variety of WellSky solutions, including clinical, billing, training, and technical troubleshooting.\n\nWe invite you to apply today and join us in shaping the future of healthcare!\n\nKey Responsibilities:\n\nReview, troubleshoot, and document escalated application technical issues.\nAssist engineering and quality assurance teams to identify, debug, test, and resolve client-facing application issues.\nDocument Salesforce cases to accurately reflect complete investigation steps taken with resulting outcomes, next steps, and resolutions while including details of all client communications.\nEscalate and document unresolved issues as required and maintain ownership of issue through resolution.\nEnsure required response time and resol

In [62]:
sponsorship = ['sponsorship','visa']
yes_sponsor = ['will consider sponsor','will sponsor']
citizen = ['US citizen, U.S. citizen', 'clearance']
sponsorship_pattern = '|'.join(sponsorship)
citizen_pattern = '|'.join(citizen)
yes_pattern = '|'.join(yes_sponsor)

In [55]:
re.search(sponsorship_pattern, test, re.IGNORECASE)

<re.Match object; span=(0, 5), match='About'>

In [58]:
for job in analyst_list:
    print(re.search(citizen_pattern, job['description'], re.IGNORECASE))

<re.Match object; span=(1841, 1848), match='citizen'>
None
<re.Match object; span=(2885, 2892), match='citizen'>
None
None
None
None
None
None
None
None
None


In [61]:
print(analyst_list[2]['description'])

About the job
Data Analyst - IT III
Chicago, IL (Day one onsite Hybrid)
10+ Months

Project Name: Secure Mobile Program

Top 3 skill sets required for this role:
 1. Use statistical tools to identify, analyze, and interpret patterns and trends in complex data sets.
 2.Collect, analyze, and interpret complex datasets to identify trends, patterns, and insights.
 3.Collect, clean, and analyze datasets to provide actionable insights.

Nice to have skills or certifications:
1. Google Advanced Data Analytics
2. Scrum Master
3. Power Bi & Excel
 Role and Responsibilities:
United Airlines is seeking a highly organized and analytical individual to join our team as a Data Analyst. This hybrid role combines data analysis expertise with project coordination skills, enabling you to drive data-driven decisions and ensure successful project execution.
 Key Responsibilities:
• Collect, analyze, and interpret complex datasets to identify trends, patterns, and insights.
• Develop and maintain comprehens

In [ ]:
def format_job_title(job_title):
    words = job_title.split()
    formatted_title = words[0]  # Keep the first word as is
    for word in words[1:]:
        formatted_title += "%20" + word  # Add "%20" before each subsequent word
    return formatted_title

job_search = format_job_title(job_title)

driver = webdriver.Chrome()

# Open LinkedIn login page
driver.get("https://www.linkedin.com/login")

# Wait for the login page to load
try:
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "rememberMeOptIn-checkbox")))

    checkbox = driver.find_element(By.ID, "rememberMeOptIn-checkbox")

    # Check if it is already selected
    # Use JavaScript to set the 'checked' property to false
    driver.execute_script("arguments[0].checked = false;", checkbox)
except:
    pass

# Enter your login credentials
username = driver.find_element(By.ID, "username")
password = driver.find_element(By.ID, "password")
username.send_keys(id)  # Replace with your LinkedIn email
password.send_keys(pw)          # Replace with your LinkedIn password

# Click the login button
password.send_keys(Keys.RETURN)

# Wait for redirection after login
WebDriverWait(driver, 10).until(EC.url_contains("https://www.linkedin.com/feed/"))

if period == "today":
    time_filter = 'r86400'
elif period == 'past week':
    time_filter = 'r604800'

analyst_list = []
scientist_list = []


search_url = f"https://www.linkedin.com/jobs/search/?f_TPR={time_filter}&keywords={job_search}&origin=JOB_SEARCH_PAGE_JOB_FILTER&sortBy=DD&start=0"

driver.get(search_url)

In [15]:
import re

parent_list = WebDriverWait(driver, 20).until(
    EC.presence_of_element_located((By.XPATH, "//*[@id='main']/div/div[2]/div[1]/div"))
)

job_listings = parent_list.find_elements(By.XPATH, "./*/li")  # Only direct children

job = job_listings[1]

job.click()

title = WebDriverWait(job, 20).until(
    EC.presence_of_element_located((By.TAG_NAME, "strong"))
).text

WebDriverWait(job, 20).until(
            EC.presence_of_element_located((By.XPATH, '//*[@id="main"]/div/div[2]/div[2]/div/div[2]/div/div/div[1]/div/div[4]/article'))
        ).text

if not re.search(top_pattern, title, re.IGNORECASE):
    if re.search(analyst_pattern, title, re.IGNORECASE):

        data = get_job_details(job)
        print(data['job_title'])
        analyst_list.append(data)

    elif re.search(scientist_pattern, title, re.IGNORECASE):
        if not re.search(senior_pattern, title, re.IGNORECASE):

            data = get_job_details(job)
            print(data['job_title'])
            scientist_list.append(data)

    else:
        pass


Fundamental Analyst - Alternative Data Analyst


In [ ]:

for job in job_listings:
    
    job.click()

    # top.find_element(By.TAG_NAME, "a").get_adttribute("text")
    # title = job.find_element(By.TAG_NAME, "strong").text
    # Search for the pattern in the text
    title = WebDriverWait(job, 20).until(
        EC.presence_of_element_located((By.TAG_NAME, "strong"))
    ).text


    if not re.search(top_pattern, title, re.IGNORECASE):
        if re.search(analyst_pattern, title, re.IGNORECASE):

            data = get_job_details(job)
            print(data['job_title'])
            analyst_list.append(data)

        elif re.search(scientist_pattern, title, re.IGNORECASE):
            if not re.search(senior_pattern, title, re.IGNORECASE):

                data = get_job_details(job)
                print(data['job_title'])
                scientist_list.append(data)

        else:
            pass

In [ ]:
for i in range(2,4):

    parent_list = WebDriverWait(driver, 20).until(
        EC.presence_of_element_located((By.XPATH, "//*[@id='main']/div/div[2]/div[1]/div"))
    )

    job_listings = parent_list.find_elements(By.XPATH, "./*/li")  # Only direct children

    for job in job_listings:
        
        job.click()

        # top.find_element(By.TAG_NAME, "a").get_adttribute("text")
        # title = job.find_element(By.TAG_NAME, "strong").text
        # Search for the pattern in the text
        title = WebDriverWait(job, 20).until(
            EC.presence_of_element_located((By.TAG_NAME, "strong"))
        ).text


        if not re.search(top_pattern, title, re.IGNORECASE):
            if re.search(analyst_pattern, title, re.IGNORECASE):

                data = get_job_details(job)
                print(data['job_title'])
                analyst_list.append(data)

            elif re.search(scientist_pattern, title, re.IGNORECASE):
                if not re.search(senior_pattern, title, re.IGNORECASE):

                    data = get_job_details(job)
                    print(data['job_title'])
                    scientist_list.append(data)

            else:
                pass

    page_button = WebDriverWait(driver, 20).until(
        EC.element_to_be_clickable((By.CSS_SELECTOR, f"button[aria-label='Page {i}']"))
    )

    # page_button = driver.find_element(By.CSS_SELECTOR, f"button[aria-label='Page {i}']")
    page_button.click()

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=132.0.6834.160)
Stacktrace:
	GetHandleVerifier [0x00007FF79A1402F5+28725]
	(No symbol) [0x00007FF79A0A2AE0]
	(No symbol) [0x00007FF799F3510A]
	(No symbol) [0x00007FF799F0EEA5]
	(No symbol) [0x00007FF799FB6F87]
	(No symbol) [0x00007FF799FCFA52]
	(No symbol) [0x00007FF799FAFD53]
	(No symbol) [0x00007FF799F7A0E3]
	(No symbol) [0x00007FF799F7B471]
	GetHandleVerifier [0x00007FF79A46F30D+3366989]
	GetHandleVerifier [0x00007FF79A4812F0+3440688]
	GetHandleVerifier [0x00007FF79A4778FD+3401277]
	GetHandleVerifier [0x00007FF79A20AAAB+858091]
	(No symbol) [0x00007FF79A0AE74F]
	(No symbol) [0x00007FF79A0AA304]
	(No symbol) [0x00007FF79A0AA49D]
	(No symbol) [0x00007FF79A098B69]
	BaseThreadInitThunk [0x00007FFE9D717374+20]
	RtlUserThreadStart [0x00007FFE9E9BCC91+33]


In [78]:
test = job_listings[0]

# top.find_element(By.TAG_NAME, "a").get_adttribute("text")
title = test.find_element(By.TAG_NAME, "strong").text
company = test.find_element(By.CLASS_NAME, "artdeco-entity-lockup__subtitle").find_element(By.TAG_NAME, "span").text
where = test.find_element(By.CLASS_NAME, "artdeco-entity-lockup__caption").find_element(By.TAG_NAME, "span").text
try:
    easy_apply = test.find_element(By.CLASS_NAME, "job-card-list__footer-wrapper").find_element(By.TAG_NAME, "span").text
    if easy_apply != 'EasyApply':
        easy_apply = "Apply"
except:
    easy_apply = "Apply"


link_block = driver.find_element(By.CLASS_NAME, "jobs-details__main-content")
link_area = link_block.find_element(By.CSS_SELECTOR, "div.t-24.job-details-jobs-unified-top-card__job-title")
link = link_area.find_element(By.TAG_NAME, "a").get_attribute("href")

job_description = driver.find_element(By.CLASS_NAME, "jobs-description__container").text
# detail = job_description.find_elements(By.TAG_NAME, "span")



In [79]:
print(f"Title : {title}")
print(f"company : {company}")
print(f"where : {where}")
print(f"Link : ", link)
print(f"EasyApply : {easy_apply}")
print("")
print(job_description)

Title : Machine Learning Engineer II, ML Fraud
company : Affirm
where : Richmond, VA (Remote)
Link :  https://www.linkedin.com/jobs/view/4058671702/?alternateChannel=search&refId=FW2XrlzYXC3AWj78kx4U4w%3D%3D&trackingId=OduRPKZ6Iy9xRRO9LTkmfA%3D%3D&trk=d_flagship3_search_srp_jobs
EasyApply : Apply

About the job
Affirm is reinventing credit to make it more honest and friendly, giving consumers the flexibility to buy now and pay later without any hidden fees or compounding interest.

Join Affirm’s ML Fraud team as a Machine Learning Engineer II and advance our capabilities in detecting and preventing online fraud. Leverage cutting-edge algorithms and proprietary data to outsmart fraudsters while ensuring seamless experiences for genuine customers. Your work will directly impact Affirm's financial performance by developing sophisticated models for real time transaction decisions.

Collaborate with experts to tackle complex challenges, create innovative solutions for emerging fraud pattern

About the job
Affirm is reinventing credit to make it more honest and friendly, giving consumers the flexibility to buy now and pay later without any hidden fees or compounding interest.

Join Affirm’s ML Fraud team as a Machine Learning Engineer II and advance our capabilities in detecting and preventing online fraud. Leverage cutting-edge algorithms and proprietary data to outsmart fraudsters while ensuring seamless experiences for genuine customers. Your work will directly impact Affirm's financial performance by developing sophisticated models for real time transaction decisions.

Collaborate with experts to tackle complex challenges, create innovative solutions for emerging fraud patterns, and advance our fraud detection capabilities. If you're passionate about machine learning, thrive on challenges, and want to make a tangible difference in fintech, Affirm is your opportunity. Help us create honest financial products that improve lives while growing your career in a dynamic, impa

In [ ]:
import openai

api = os.getenv("OPENAI_API_KEY")

openai.api_key = api

# rating : Out of 0~10 how well does the job description match my resume.

In [102]:
system_prompt = f"""
You are a job searching assistant
You will extract information from a given a job posting and output it in json format.

Here are the variables you will be looking for. If you are unable to find information, return N/A: 
sponsorship : is sponsorship available for the position. [yes, no, N/A]
citizenship : is US citizenship required for the position. [yes, no, N/A]
clearance : is clearance required for the position. [yes, no, N/A]
education : minimum degree of education [bachelors, masters, phd, N/A]

Job Posting : 
{job_description.text}


"""

In [144]:
system_prompt = f"""
You are a job searching assistant.
You will extract information from a given a job posting and output it in json format.
Only output the json and not around code blocks

Here are the variables you will be looking for. 
If the job description does not specify information, return N/A: 

sponsorship : is sponsorship available for the position. [yes, no, N/A]
citizenship : is US citizenship required for the position. [yes, no, N/A]
clearance : is clearance required for the position. [yes, no, N/A]
education : minimum degree of education [bachelors, masters, phd, N/A]
experience : what is the minimum required years of work experience [integer]
"""

In [154]:
system_prompt = """
You are an AI assistant specializing in extracting structured information from job postings. 
Return a JSON object with the following fields but only output the json data and don't output the code blocks.:


sponsorship: Indicates whether visa sponsorship is available for the position. Possible values: "yes", "no", or "N/A" (if not specified).
citizenship: Specifies if U.S. citizenship is required. Possible values: "yes", "no", or "N/A" (if not mentioned).
clearance: Indicates whether a security clearance is required. Possible values: "yes", "no", or "N/A".
education: The minimum required degree level. Possible values: "bachelors", "masters", "phd", or "N/A" if not provided.
experience: The minimum required years of work experience as an integer. If not mentioned, return "N/A".
"""

In [160]:
system_prompt = """
You are an AI assistant specializing in extracting structured information from job postings.
If information is not specified, do not answer 'no' but answer N/A.
Return a JSON object with the following fields but only output the json data and don't output the code blocks:


sponsorship: Does the company provide visa sponsorship? Possible values: "yes", "no", or "N/A" (if not specified).
citizenship: Specifies if U.S. citizenship is required. Possible values: "yes", "no", or "N/A" (if not mentioned).
clearance: Indicates whether a security clearance is required. Possible values: "yes", "no", or "N/A".
education: The minimum required degree level. Possible values: "bachelors", "masters", "phd", or "N/A" if not provided.
experience: The minimum required years of work experience as an integer. If not mentioned, return "N/A".
"""

In [161]:
client = openai.OpenAI()

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": job_description.text}
    ],
    temperature=0,
    max_tokens=150
)
answer = response.choices[0].message.content

In [162]:
print(answer)

{
  "sponsorship": "N/A",
  "citizenship": "N/A",
  "clearance": "N/A",
  "education": "masters",
  "experience": 3
}


In [120]:
import json
answer_dict = json.loads(answer)
answer_dict

{'sponsorship': 'no',
 'citizenship': 'N/A',
 'clearance': 'N/A',
 'education': 'masters'}

In [153]:
print(job_description.text)

About the job
Location: Dallas, TX

This position will be Hybrid.

This role will not support relocation.

Summary:

This role within our Information Services department is looking for a Data Scientist that can apply scientific principles to analyze complex datasets across different groups to make business decisions. Develops scalable algorithms and models.

Major Responsibilities: 

Translates well-structured business problems into data science problems. Analyzes complex datasets to inform or make decisions to address those business problems. 
Applies scientific principles and concepts to support invention. Uses a range of scientific methodologies and follows best practices for those methodologies. 
Proactively identifies solutions to problems. Solutions may require novel techniques or approaches. 
Creates metrics and measurements to quantify the business impact of the work. 
Writes accurate and clear technical documentation and reports, including appropriate mathematical rigor as nee

In [31]:
# button = driver.find_element(By.ID, "searchFilter_timePostedRange")
# button.click()

In [32]:
for i,j in enumerate(job_listings):
    print(j.text)
    print(i)
    print("")

Applied Machine Learning Engineers
Applied Machine Learning Engineers with verification
Shopify
NAMER (Remote)
12 school alumni work here
12 Georgetown University school alumni work here
Viewed
0

Data Scientist Intern
Data Scientist Intern
Relias
Morrisville, NC (Hybrid)
1 school alum works here
1 Georgetown University school alum works here
23 hours ago
Within the past 24 hours
1

Decision Scientist - Trend and UM Strategy and Management
Decision Scientist - Trend and UM Strategy and Management with verification
CVS Health
Springfield, IL (Remote)
$72.1K/yr - $173K/yr · 401(k) benefit
26 company alumni work here
26 Tractor Supply Company company alumni work here
40 minutes ago
Within the past 24 hours
2

Sr ML Research Data Scientist
Sr ML Research Data Scientist
WalkWater Technologies
Seattle, WA (Hybrid)
Promoted
Easy Apply
3

AI/Machine Learning SME
AI/Machine Learning SME with verification
ProSearch
Greater Boston (Remote)
Response time is typically 4 days
Promoted
Easy Apply
4



In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os
import glob
import pandas as pd

# Get all CSV files in the current directory
csv_files = glob.glob("*.csv")

# Sort files by filename in descending order
csv_files_sorted = sorted(csv_files, reverse=True)

if csv_files_sorted:
    latest_csv = csv_files_sorted[0]
    current = pd.read_csv(f'./{latest_csv}')
else:
    current = pd.DataFrame()

# current = pd.DataFrame()

# Initialize an empty DataFrame to store job data
# job_data_list = []

job_data = pd.DataFrame(columns=["Job Title", "Link", "When", "Company", "Location"])



# Set up the Selenium WebDriver (replace the path with your ChromeDriver path)
driver = webdriver.Chrome()

# try:
# Open LinkedIn login page
driver.get("https://www.linkedin.com/login")

# Wait for the login page to load
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.ID, "rememberMeOptIn-checkbox")))

checkbox = driver.find_element(By.ID, "rememberMeOptIn-checkbox")

# Check if it is already selected
# Use JavaScript to set the 'checked' property to false
driver.execute_script("arguments[0].checked = false;", checkbox)

# Enter your login credentials
username = driver.find_element(By.ID, "username")
password = driver.find_element(By.ID, "password")
username.send_keys(id)  # Replace with your LinkedIn email
password.send_keys(pw)          # Replace with your LinkedIn password

# Click the login button
password.send_keys(Keys.RETURN)

# Wait for redirection after login
WebDriverWait(driver, 10).until(EC.url_contains("https://www.linkedin.com/feed/"))

num = 0

for i in range(0,1000,10):

    try:
        # Navigate to the saved jobs page
        driver.get(f"https://www.linkedin.com/my-items/saved-jobs/?cardType=APPLIED&start={i}")

        # Wait for the saved jobs page to load
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "ul")))

        results_container = driver.find_element(By.CLASS_NAME, "workflow-results-container")
        block = results_container.find_element(By.CLASS_NAME, "list-style-none")
        blocks = block.find_elements(By.CLASS_NAME, "linked-area")

        for job in blocks:
            try:
                top = job.find_element(By.CLASS_NAME, "mb1")
                raw_string = top.find_element(By.TAG_NAME, "a").get_attribute("text")
                title = ' '.join(raw_string.split()).strip()
                link = top.find_element(By.TAG_NAME, "a").get_attribute("href")
                link = link.split("?refId")[0]
                when = job.find_element(By.CLASS_NAME, "entity-result__insights").find_element(By.TAG_NAME, "span").text

                temp = top.find_elements(By.CLASS_NAME, "t-14")
                company = temp[0].text
                location = temp[1].text

                if (current['JobLinks'] == link).any() and "viewed" not in when:
                    driver.quit()
                    break

                print(f"Job Title: {title}")
                print(f"Link: {link}")
                print(when)
                print(company)
                print(location)
                print("")

                num += 1
                print(num)

                # Append data to the DataFrame
                row_data = {
                    "Job Title": title,
                    "Link": link,
                    "When": when,
                    "Company": company,
                    "Location": location
                }

                # job_data = pd.concat([pd.DataFrame([row_data]), job_data], ignore_index=True)
                job_data = pd.concat([job_data, pd.DataFrame([row_data])], ignore_index=True)

            except Exception as e:
                print("Error scraping job:", e)

    except:
        driver.quit()
        break

driver.quit()


Job Title: AI Engineer , Verified
Link: https://www.linkedin.com/jobs/view/4143949417/
Applied 1m ago
LinkedIn
Sunnyvale, CA (Hybrid)

1
Job Title: Machine Learning Engineer , Verified
Link: https://www.linkedin.com/jobs/view/4143654892/
Resume downloaded 6h ago
INSPYR Solutions
Cupertino, CA (Hybrid)

2
Job Title: Data Scientist , Verified
Link: https://www.linkedin.com/jobs/view/4142694570/
Applied 6h ago
Gotham Technology Group
Somerset County, NJ (Hybrid)

3
Job Title: Data Scientist- Real Estate , Verified
Link: https://www.linkedin.com/jobs/view/4084703890/
Applied 6h ago
Circle K
Tempe, AZ

4
Job Title: Data Scientist, Growth , Verified
Link: https://www.linkedin.com/jobs/view/4143308442/
Applied on Company Website 6h ago
Notion
New York, NY (On-site)

5
Job Title: Data Scientist , Verified
Link: https://www.linkedin.com/jobs/view/4066513761/
Applied on Company Website 7h ago
Nirvana
Austin, TX (Hybrid)

6
Job Title: Data Scientist with AI
Link: https://www.linkedin.com/jobs/vie

In [2]:
job_data.shape

(95, 5)

In [3]:
import pandas as pd
from datetime import datetime, timedelta
import re

# Define a function to parse the "When" column and calculate the applied date
def calculate_applied_date(row, current_time):
    match = re.search(r'(\d+)([a-zA-Z]+) ago', row)
    if match:
        value, unit = int(match.group(1)), match.group(2)
        if unit == 'h':  # hours
            date = current_time - timedelta(hours=value)
        elif unit == 'd':  # days
            date = current_time - timedelta(days=value)
        elif unit == 'w':  # weeks
            date = current_time - timedelta(weeks=value)
        elif unit == 'm':  # minutes
            date = current_time - timedelta(minutes=value)
        elif unit == 'mo':  # months (approximate as 30 days per month)
            date = current_time - timedelta(days=value * 30)
        else:
            return None
        return date.strftime('%Y-%m-%d')  # Format as yyyy-mm-dd
    return None
# Calculate the "applied_date" column
current_time = datetime.now()
job_data['applied_date'] = job_data['When'].apply(lambda x: calculate_applied_date(x, current_time))
job_data = job_data[~job_data['When'].str.contains('viewed', case=False, na=False)]

def assign_application_source(row):
    if 'Applied on Company Website' in row:
        return 'LinkedIn'
    else:
        return 'EasyApply'

# Apply the function to create a new column
job_data['Source'] = job_data['When'].apply(assign_application_source)

# Categorization function
def categorize_job(title):
    title = title.lower()  # Convert to lowercase for case-insensitive matching
    if 'analyst' in title or 'analytics' in title :
        return 'Analyst'
    elif 'scientist' in title or 'science' in title:
        return 'Scientist'
    elif 'ai' in title or 'ml' in title or 'engineer' in title or 'machine' in title:
        return 'AI'
    return 'Other'

# Apply categorization
job_data['JobTitle'] = job_data['Job Title'].str.replace(', Verified', '', regex=False)
job_data['Category'] = job_data['JobTitle'].apply(categorize_job)
job_data['JobLinks'] = job_data['Link'].str.split("\\?refId").str[0]

output = job_data[~job_data['applied_date'].isnull()]
output = output[['Source','JobTitle','Category','Company','Location','JobLinks','applied_date']]
output

,Source,JobTitle,Category,Company,Location,JobLinks,applied_date
0,EasyApply,AI Engineer,AI,LinkedIn,"Sunnyvale, CA (Hybrid)",https://www.linkedin.com/jobs/view/4143949417/,2025-02-04
1,EasyApply,Machine Learning Engineer,AI,INSPYR Solutions,"Cupertino, CA (Hybrid)",https://www.linkedin.com/jobs/view/4143654892/,2025-02-04
2,EasyApply,Data Scientist,Scientist,Gotham Technology Group,"Somerset County, NJ (Hybrid)",https://www.linkedin.com/jobs/view/4142694570/,2025-02-04
3,EasyApply,Data Scientist- Real Estate,Scientist,Circle K,"Tempe, AZ",https://www.linkedin.com/jobs/view/4084703890/,2025-02-04
4,LinkedIn,"Data Scientist, Growth",Scientist,Notion,"New York, NY (On-site)",https://www.linkedin.com/jobs/view/4143308442/,2025-02-04
...,...,...,...,...,...,...,...
90,EasyApply,Azure AI Engineer,AI,Tenth Revolution Group,New York City Metropolitan Area (Hybrid),https://www.linkedin.com/jobs/view/4135543836/,2025-02-01
91,EasyApply,Senior AI Engineer,AI,Palo Alto Networks,"Santa Clara, CA (On-site)",https://www.linkedin.com/jobs/view/4137503217/,2025-02-01
92,LinkedIn,Samsung Ads - Machine Learning Model Engineer,AI,Samsung Electronics America,"Mountain View, CA (Hybrid)",https://www.linkedin.com/jobs/view/4119018263/,2025-02-01
93,LinkedIn,"2025 Summer Intern, Web AI Agent Research Engi...",AI,Samsung Research America (SRA),"Mountain View, CA (On-site)",https://www.linkedin.com/jobs/view/4138819054/,2025-02-01


In [4]:
from datetime import datetime

final = pd.concat([output, current], ignore_index=True)
# final = pd.concat([output], ignore_index=True)

# Get current date and format it
current_date = datetime.now().strftime('%Y%m%d')

final.to_csv(f'./linkedin_application_{current_date}.csv', index=False)